In [1]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
import xgboost as xgb
import optuna
import numpy as np

# Set the Optuna logger to output only WARNING and higher levels
optuna.logging.set_verbosity(optuna.logging.WARNING)

sns.set_theme()
sns.set_context("notebook")
%load_ext autoreload
%autoreload 2

/Users/user/anaconda3/envs/GIGACOW/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dtype_dict = {
    'Date': 'str',
    'FarmName_Pseudo': 'str',
    'SE_Number': 'str',
    'Age': 'Int64',
    'BreedName': 'str',
    'DailyYield': 'float',
    'PreviousDailyYield': 'float',
    'DailyYieldChange': 'float',
    'DaysInMilk': 'Int64',
    'YearSeason': 'str',
    'LactationNumber': 'Int64',
    'ExpectedYield': 'float',
    'NormalizedDailyYield': 'float',
    'NormalizedDailyYieldChange': 'float',
    'HeatStress': 'Int64',
    'Temp15Threshold': 'Int64',
    'HW': 'Int64',
    'cum_HW': 'Int64',
    'MeanTemperature': 'float',
    'MeanTHI_adj': 'float',
    'HeatLoad': 'float',
    'CumulativeHeatLoad': 'float',
}

milk_data = pd.read_csv('../Data/MergedData/HeatApproachYieldDataTest.csv', dtype=dtype_dict)
milk_data['Date'] = pd.to_datetime(milk_data['Date'], format='%Y-%m-%d')
milk_data.head(-5)

,Date,FarmName_Pseudo,SE_Number,Age,LactationNumber,DaysInMilk,YearSeason,DailyYield,PreviousDailyYield,DailyYieldChange,ExpectedYield,NormalizedDailyYield,NormalizedDailyYieldChange,HeatStress,Temp15Threshold,HW,cum_HW,MeanTemperature,MeanTHI_adj
0,2022-01-01,a624fb9a,SE-064c0cec-1189,3095,7,191,2022-1,30.77,0.00,0.00,29.739372,1.034655,0.000000,0,0,0,0,-3.025000,28.012944
1,2022-01-02,a624fb9a,SE-064c0cec-1189,3096,7,192,2022-1,48.22,30.77,17.45,29.692059,1.624003,0.587699,0,0,0,0,-0.279167,32.898193
2,2022-01-03,a624fb9a,SE-064c0cec-1189,3097,7,193,2022-1,30.53,48.22,-17.69,29.644756,1.029862,-0.596733,0,0,0,0,2.033333,36.760487
3,2022-01-04,a624fb9a,SE-064c0cec-1189,3098,7,194,2022-1,42.26,30.53,11.73,29.597463,1.427825,0.396318,0,0,0,0,0.066667,31.939524
4,2022-01-05,a624fb9a,SE-064c0cec-1189,3099,7,195,2022-1,38.49,42.26,-3.77,29.550181,1.302530,-0.127580,0,0,0,0,-3.700000,26.498206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487083,2023-06-03,f454e660,SE-fcdf259d-0044-0,4150,10,347,2023-3,12.67,15.75,-3.08,11.811455,1.072687,-0.260764,0,1,0,0,12.666667,53.132530
487084,2023-06-04,f454e660,SE-fcdf259d-0044-0,4151,10,348,2023-3,22.31,12.67,9.64,11.750492,1.898644,0.820391,0,1,0,0,13.079167,56.726870
487085,2023-06-05,f454e660,SE-fcdf259d-0044-0,4152,10,349,2023-3,12.84,22.31,-9.47,11.689810,1.098393,-0.810107,0,1,0,0,14.237500,58.482418
487086,2023-06-06,f454e660,SE-fcdf259d-0044-0,4153,10,350,2023-3,9.47,12.84,-3.37,11.629408,0.814315,-0.289783,1,1,0,0,15.345833,60.546358


In [3]:
# Initialize an empty DataFrame to store results
results_df = pd.DataFrame(columns=['FarmName_Pseudo', 'FarmHeatStressProduction'])

# Specify the farm ID for analysis
farm_id = 'a624fb9a'

# Filter data for the specific farm
farm_data = milk_data[milk_data['FarmName_Pseudo'] == farm_id]

# Check if NormalizedDailyYield is centered around 1
normalized_mean = farm_data['NormalizedDailyYield'].mean()
normalized_variance = farm_data['NormalizedDailyYield'].var()
print("Mean of NormalizedDailyYield:", normalized_mean)
print("Standard Deviation of NormalizedDailyYield:", farm_data['NormalizedDailyYield'].std())
print("Variance of NormalizedDailyYield:", normalized_variance)

# Define the target variable
target = 'NormalizedDailyYield'

# Split the data into train and validation sets
train_data, val_data = train_test_split(farm_data, test_size=0.3, random_state=42)

# Define the single feature
features = ['HeatStress']

# Function to optimize hyperparameters using Optuna
def objective(trial):
    param = {
        'verbosity': 0,
        'objective': 'reg:squarederror',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 1, 15),
    }

    # Cross-validation to estimate performance
    model = xgb.XGBRegressor(**param)
    scores = cross_val_score(model, train_data[features], train_data[target], cv=5, scoring='neg_mean_squared_error')
    mse = -scores.mean()
    return mse

# Function to fit the model and calculate reduction due to HeatStress
def fit_xgboost_and_calculate_reduction(train_data, val_data, features, baseline_mean):
    print(f"\nSelected features: {features}")

    # Optimize hyperparameters using Optuna
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100, timeout=600)
    best_params = study.best_params
    print(f"Best parameters: {best_params}")

    # Train final model using the best parameters
    best_model = xgb.XGBRegressor(**best_params)
    best_model.fit(train_data[features], train_data[target])

    # Predict the NormalizedDailyYield under heat stress (HeatStress = 1)
    val_data_heatstress = val_data.copy()
    val_data_heatstress['HeatStress'] = 1
    y_pred_heatstress = best_model.predict(val_data_heatstress[features])
    avg_production_heatstress = y_pred_heatstress.mean()

    # Predict the NormalizedDailyYield without heat stress (HeatStress = 0)
    val_data_no_heatstress = val_data.copy()
    val_data_no_heatstress['HeatStress'] = 0
    y_pred_no_heatstress = best_model.predict(val_data_no_heatstress[features])
    avg_production_no_heatstress = y_pred_no_heatstress.mean()

    # Calculate the reduction due to heat stress
    reduction_due_to_heatstress = avg_production_heatstress - avg_production_no_heatstress

    print(f"Estimated average production during heat stress: {avg_production_heatstress:.4f}")
    print(f"Estimated average production without heat stress: {avg_production_no_heatstress:.4f}")
    print(f"Reduction in milk production during heat stress: {reduction_due_to_heatstress:.4f}")

    return reduction_due_to_heatstress

# Train the model and calculate the reduction
reduction = fit_xgboost_and_calculate_reduction(train_data, val_data, features, normalized_mean)

# Create a new DataFrame for the current farm's result
new_result = pd.DataFrame([{
    'FarmName_Pseudo': farm_id,
    'FarmHeatStressProduction': reduction
}])

# Check if results_df is empty before concatenation
if results_df.empty:
    results_df = new_result
else:
    results_df = pd.concat([results_df, new_result], ignore_index=True)

results_df

Baseline mean of NormalizedDailyYield for all days: 0.9969

Selected features: ['MeanTemperature', 'HW']
Best parameters: {'lambda': 0.9127424480338338, 'alpha': 1.0930017110551124e-08, 'subsample': 0.5948725117177914, 'colsample_bytree': 0.6861993477337037, 'learning_rate': 0.08484572884822209, 'n_estimators': 58, 'max_depth': 10}
Final model performance: MSE = 0.0423 ± 0.0011
Mean Squared Error on test set: 0.0397
Estimated average milk production during heat stress: 0.9780
Reduction in milk production during heat stress: 1.90%

Selected features: ['MeanTemperature', 'cum_HW']
Best parameters: {'lambda': 0.03098441945475088, 'alpha': 0.04863761065304875, 'subsample': 0.9769860827552345, 'colsample_bytree': 0.8413537917565252, 'learning_rate': 0.09536446968114216, 'n_estimators': 216, 'max_depth': 5}
Final model performance: MSE = 0.0423 ± 0.0011
Mean Squared Error on test set: 0.0396
Estimated average milk production during heat stress: 0.9781
Reduction in milk production during heat

In [5]:
# Specify the farm ID for analysis
farm_id = '5c06d92d'

# Filter data for the specific farm
farm_data = milk_data[milk_data['FarmName_Pseudo'] == farm_id]

# Check if NormalizedDailyYield is centered around 1
normalized_mean = farm_data['NormalizedDailyYield'].mean()
normalized_variance = farm_data['NormalizedDailyYield'].var()
print("Mean of NormalizedDailyYield:", normalized_mean)
print("Standard Deviation of NormalizedDailyYield:", farm_data['NormalizedDailyYield'].std())
print("Variance of NormalizedDailyYield:", normalized_variance)

# Define the target variable
target = 'NormalizedDailyYield'

# Split the data into train and validation sets
train_data, val_data = train_test_split(farm_data, test_size=0.3, random_state=42)

# Define the single feature
features = ['HeatStress']

# Function to optimize hyperparameters using Optuna
def objective(trial):
    param = {
        'verbosity': 0,
        'objective': 'reg:squarederror',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 1, 15),
    }

    # Cross-validation to estimate performance
    model = xgb.XGBRegressor(**param)
    scores = cross_val_score(model, train_data[features], train_data[target], cv=5, scoring='neg_mean_squared_error')
    mse = -scores.mean()
    return mse

# Function to fit the model and calculate reduction due to HeatStress
def fit_xgboost_and_calculate_reduction(train_data, val_data, features, baseline_mean):
    print(f"\nSelected features: {features}")

    # Optimize hyperparameters using Optuna
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100, timeout=600)
    best_params = study.best_params
    print(f"Best parameters: {best_params}")

    # Train final model using the best parameters
    best_model = xgb.XGBRegressor(**best_params)
    best_model.fit(train_data[features], train_data[target])

    # Predict the NormalizedDailyYield under heat stress (HeatStress = 1)
    val_data_heatstress = val_data.copy()
    val_data_heatstress['HeatStress'] = 1
    y_pred_heatstress = best_model.predict(val_data_heatstress[features])
    avg_production_heatstress = y_pred_heatstress.mean()

    # Predict the NormalizedDailyYield without heat stress (HeatStress = 0)
    val_data_no_heatstress = val_data.copy()
    val_data_no_heatstress['HeatStress'] = 0
    y_pred_no_heatstress = best_model.predict(val_data_no_heatstress[features])
    avg_production_no_heatstress = y_pred_no_heatstress.mean()

    # Calculate the reduction due to heat stress
    reduction_due_to_heatstress = avg_production_heatstress - avg_production_no_heatstress

    print(f"Estimated average production during heat stress: {avg_production_heatstress:.4f}")
    print(f"Estimated average production without heat stress: {avg_production_no_heatstress:.4f}")
    print(f"Reduction in milk production during heat stress: {reduction_due_to_heatstress:.4f}")

    return reduction_due_to_heatstress

# Train the model and calculate the reduction
reduction = fit_xgboost_and_calculate_reduction(train_data, val_data, features, normalized_mean)

# Create a new DataFrame for the current farm's result
new_result = pd.DataFrame([{
    'FarmName_Pseudo': farm_id,
    'FarmHeatStressProduction': reduction
}])

# Check if results_df is empty before concatenation
if results_df.empty:
    results_df = new_result
else:
    results_df = pd.concat([results_df, new_result], ignore_index=True)

results_df

Baseline mean of NormalizedDailyYield for all days: 0.9960

Selected features: ['MeanTemperature', 'HW']
Best parameters: {'lambda': 2.6466616258787035e-06, 'alpha': 0.0006916141663389971, 'subsample': 0.964093668262603, 'colsample_bytree': 0.7288410058524772, 'learning_rate': 0.020523306990280262, 'n_estimators': 677, 'max_depth': 10}
Final model performance: MSE = 0.0274 ± 0.0014
Mean Squared Error on test set: 0.0281
Estimated average milk production during heat stress: 1.0020
Reduction in milk production during heat stress: -0.61%

Selected features: ['MeanTemperature', 'cum_HW']
Best parameters: {'lambda': 4.0778634872480876e-05, 'alpha': 0.014217740830501977, 'subsample': 0.6945667793762321, 'colsample_bytree': 0.7059992215361837, 'learning_rate': 0.030370418646064216, 'n_estimators': 241, 'max_depth': 13}
Final model performance: MSE = 0.0274 ± 0.0014
Mean Squared Error on test set: 0.0281
Estimated average milk production during heat stress: 1.0020
Reduction in milk production 

In [7]:
# Specify the farm ID for analysis
farm_id = '752efd72'

# Filter data for the specific farm
farm_data = milk_data[milk_data['FarmName_Pseudo'] == farm_id]

# Check if NormalizedDailyYield is centered around 1
normalized_mean = farm_data['NormalizedDailyYield'].mean()
normalized_variance = farm_data['NormalizedDailyYield'].var()
print("Mean of NormalizedDailyYield:", normalized_mean)
print("Standard Deviation of NormalizedDailyYield:", farm_data['NormalizedDailyYield'].std())
print("Variance of NormalizedDailyYield:", normalized_variance)

# Define the target variable
target = 'NormalizedDailyYield'

# Split the data into train and validation sets
train_data, val_data = train_test_split(farm_data, test_size=0.3, random_state=42)

# Define the single feature
features = ['HeatStress']

# Function to optimize hyperparameters using Optuna
def objective(trial):
    param = {
        'verbosity': 0,
        'objective': 'reg:squarederror',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 1, 15),
    }

    # Cross-validation to estimate performance
    model = xgb.XGBRegressor(**param)
    scores = cross_val_score(model, train_data[features], train_data[target], cv=5, scoring='neg_mean_squared_error')
    mse = -scores.mean()
    return mse

# Function to fit the model and calculate reduction due to HeatStress
def fit_xgboost_and_calculate_reduction(train_data, val_data, features, baseline_mean):
    print(f"\nSelected features: {features}")

    # Optimize hyperparameters using Optuna
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100, timeout=600)
    best_params = study.best_params
    print(f"Best parameters: {best_params}")

    # Train final model using the best parameters
    best_model = xgb.XGBRegressor(**best_params)
    best_model.fit(train_data[features], train_data[target])

    # Predict the NormalizedDailyYield under heat stress (HeatStress = 1)
    val_data_heatstress = val_data.copy()
    val_data_heatstress['HeatStress'] = 1
    y_pred_heatstress = best_model.predict(val_data_heatstress[features])
    avg_production_heatstress = y_pred_heatstress.mean()

    # Predict the NormalizedDailyYield without heat stress (HeatStress = 0)
    val_data_no_heatstress = val_data.copy()
    val_data_no_heatstress['HeatStress'] = 0
    y_pred_no_heatstress = best_model.predict(val_data_no_heatstress[features])
    avg_production_no_heatstress = y_pred_no_heatstress.mean()

    # Calculate the reduction due to heat stress
    reduction_due_to_heatstress = avg_production_heatstress - avg_production_no_heatstress

    print(f"Estimated average production during heat stress: {avg_production_heatstress:.4f}")
    print(f"Estimated average production without heat stress: {avg_production_no_heatstress:.4f}")
    print(f"Reduction in milk production during heat stress: {reduction_due_to_heatstress:.4f}")

    return reduction_due_to_heatstress

# Train the model and calculate the reduction
reduction = fit_xgboost_and_calculate_reduction(train_data, val_data, features, normalized_mean)

# Create a new DataFrame for the current farm's result
new_result = pd.DataFrame([{
    'FarmName_Pseudo': farm_id,
    'FarmHeatStressProduction': reduction
}])

# Check if results_df is empty before concatenation
if results_df.empty:
    results_df = new_result
else:
    results_df = pd.concat([results_df, new_result], ignore_index=True)

results_df

Baseline mean of NormalizedDailyYield for all days: 0.9936

Selected features: ['MeanTemperature', 'HW']
Best parameters: {'lambda': 0.5271947571865087, 'alpha': 5.475606033492812e-06, 'subsample': 0.5528188421802576, 'colsample_bytree': 0.6706625164634326, 'learning_rate': 0.006507853161735247, 'n_estimators': 956, 'max_depth': 15}
Final model performance: MSE = 0.0236 ± 0.0008
Mean Squared Error on test set: 0.0227
Estimated average milk production during heat stress: 0.9810
Reduction in milk production during heat stress: 1.27%

Selected features: ['MeanTemperature', 'cum_HW']
Best parameters: {'lambda': 0.003448209474205623, 'alpha': 3.473065812157643e-06, 'subsample': 0.8290113689453642, 'colsample_bytree': 0.7213187705034583, 'learning_rate': 0.010473170658938774, 'n_estimators': 904, 'max_depth': 12}
Final model performance: MSE = 0.0235 ± 0.0008
Mean Squared Error on test set: 0.0227
Estimated average milk production during heat stress: 0.9810
Reduction in milk production durin

In [9]:
# Specify the farm ID for analysis
farm_id = 'f454e660'

# Filter data for the specific farm
farm_data = milk_data[milk_data['FarmName_Pseudo'] == farm_id]

# Check if NormalizedDailyYield is centered around 1
normalized_mean = farm_data['NormalizedDailyYield'].mean()
normalized_variance = farm_data['NormalizedDailyYield'].var()
print("Mean of NormalizedDailyYield:", normalized_mean)
print("Standard Deviation of NormalizedDailyYield:", farm_data['NormalizedDailyYield'].std())
print("Variance of NormalizedDailyYield:", normalized_variance)

# Define the target variable
target = 'NormalizedDailyYield'

# Split the data into train and validation sets
train_data, val_data = train_test_split(farm_data, test_size=0.3, random_state=42)

# Define the single feature
features = ['HeatStress']

# Function to optimize hyperparameters using Optuna
def objective(trial):
    param = {
        'verbosity': 0,
        'objective': 'reg:squarederror',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 1, 15),
    }

    # Cross-validation to estimate performance
    model = xgb.XGBRegressor(**param)
    scores = cross_val_score(model, train_data[features], train_data[target], cv=5, scoring='neg_mean_squared_error')
    mse = -scores.mean()
    return mse

# Function to fit the model and calculate reduction due to HeatStress
def fit_xgboost_and_calculate_reduction(train_data, val_data, features, baseline_mean):
    print(f"\nSelected features: {features}")

    # Optimize hyperparameters using Optuna
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100, timeout=600)
    best_params = study.best_params
    print(f"Best parameters: {best_params}")

    # Train final model using the best parameters
    best_model = xgb.XGBRegressor(**best_params)
    best_model.fit(train_data[features], train_data[target])

    # Predict the NormalizedDailyYield under heat stress (HeatStress = 1)
    val_data_heatstress = val_data.copy()
    val_data_heatstress['HeatStress'] = 1
    y_pred_heatstress = best_model.predict(val_data_heatstress[features])
    avg_production_heatstress = y_pred_heatstress.mean()

    # Predict the NormalizedDailyYield without heat stress (HeatStress = 0)
    val_data_no_heatstress = val_data.copy()
    val_data_no_heatstress['HeatStress'] = 0
    y_pred_no_heatstress = best_model.predict(val_data_no_heatstress[features])
    avg_production_no_heatstress = y_pred_no_heatstress.mean()

    # Calculate the reduction due to heat stress
    reduction_due_to_heatstress = avg_production_heatstress - avg_production_no_heatstress

    print(f"Estimated average production during heat stress: {avg_production_heatstress:.4f}")
    print(f"Estimated average production without heat stress: {avg_production_no_heatstress:.4f}")
    print(f"Reduction in milk production during heat stress: {reduction_due_to_heatstress:.4f}")

    return reduction_due_to_heatstress

# Train the model and calculate the reduction
reduction = fit_xgboost_and_calculate_reduction(train_data, val_data, features, normalized_mean)

# Create a new DataFrame for the current farm's result
new_result = pd.DataFrame([{
    'FarmName_Pseudo': farm_id,
    'FarmHeatStressProduction': reduction
}])

# Check if results_df is empty before concatenation
if results_df.empty:
    results_df = new_result
else:
    results_df = pd.concat([results_df, new_result], ignore_index=True)

results_df

Baseline mean of NormalizedDailyYield for all days: 0.9981

Selected features: ['MeanTemperature', 'HW']
Best parameters: {'lambda': 0.16498899856060845, 'alpha': 2.3940339030043374e-06, 'subsample': 0.5861392596777066, 'colsample_bytree': 0.6751801447412262, 'learning_rate': 0.00416340420815762, 'n_estimators': 663, 'max_depth': 15}
Final model performance: MSE = 0.0678 ± 0.0011
Mean Squared Error on test set: 0.0669
Estimated average milk production during heat stress: 1.0099
Reduction in milk production during heat stress: -1.17%

Selected features: ['MeanTemperature', 'cum_HW']
Best parameters: {'lambda': 1.4094873852130087e-07, 'alpha': 2.931782477570702e-07, 'subsample': 0.7823246531594824, 'colsample_bytree': 0.6432529886512759, 'learning_rate': 0.0067960880156064446, 'n_estimators': 430, 'max_depth': 15}
Final model performance: MSE = 0.0678 ± 0.0011
Mean Squared Error on test set: 0.0669
Estimated average milk production during heat stress: 1.0099
Reduction in milk production 

In [ ]:
# Merge the results_df with milk_data on 'FarmName_Pseudo'
milk_data = milk_data.merge(results_df, on='FarmName_Pseudo', how='left')

# Set 'FarmHeatStressProduction' to NaN where 'HeatStress' is 0
milk_data.loc[milk_data['HeatStress'] == 0, 'FarmHeatStressProduction'] = np.nan
milk_data.head(-5)

In [14]:
# Save the new data to a new CSV file called 'XGBMilkFarmYieldData.csv' in same folder
milk_data.to_csv('../Data/MergedData/XGBHeatFarmYieldData.csv', index=False)

### Variables Explanation for `XGBHeatFarmYieldData.csv`

1. **Date**:
   - Description: The date when the milk yield was recorded.
   - Datatype: `datetime`
   - Format: `YYYY-MM-DD`
   - Example: `2022-01-01`

2. **FarmName_Pseudo**:
   - Description: A pseudo-identifier for the farm where the data was collected.
   - Datatype: `str`
   - Example: `a624fb9a`

3. **SE_Number**:
   - Description: A unique identifier for the cow, which has been formatted to include the farm and the animal number.
   - Datatype: `str`
   - Example: `SE-064c0cec-1189`

4. **Age**:
   - Description: The age of the cow in days.
   - Datatype: `Int64`
   - Example: `3095`

5. **BreedName**:
   - Description: The breed name of the cow.
   - Datatype: `str`
   - Example: `02 SLB`

6. **LactationNumber**:
   - Description: The number assigned to the cow's lactation cycle.
   - Datatype: `Int64`
   - Example: `7`

7. **DaysInMilk**:
   - Description: The number of days the cow has been in milk (lactating) at the time of recording.
   - Datatype: `Int64`
   - Example: `191`

8. **YearSeason**:
   - Description: The seasonal period based on the year and the month range.
   - Datatype: `str`
   - Example: `2022-1`
   - YearSeason parameters in yield datasets:
     - 1: Dec-Feb
     - 2: Mar-May
     - 3: Jun-Aug
     - 4: Sep-Nov

9. **DailyYield**:
   - Description: The total amount of milk produced by the cow in a single day.
   - Datatype: `float`
   - Example: `30.77`

10. **PreviousDailyYield**:
    - Description: The total amount of milk produced by the cow on the previous day.
    - Datatype: `float`
    - Example: `0.0`

11. **DailyYieldChange**:
    - Description: The change in daily milk yield from the previous day.
    - Datatype: `float`
    - Example: `0.0`

12. **ExpectedYield**:
    - Description: The expected amount of milk yield based on certain models or predictions.
    - Datatype: `float`
    - Example: `35.914865`

13. **NormalizedDailyYield**:
    - Description: The daily yield normalized to account for various factors.
    - Datatype: `float`
    - Example: `0.856748`

14. **NormalizedDailyYieldChange**:
    - Description: The change in normalized daily yield from the previous day.
    - Datatype: `float`
    - Example: `0.0`

15. **HeatStress**:
    - Description: A binary variable indicating the presence of heat stress on the cow.
    - Datatype: `Int64`
    - Example: `0`

16. **Temp15Threshold**:
    - Description: A binary variable indicating if the temperature exceeded 15 degrees Celsius on the given day.
    - Datatype: `Int64`
    - Example: `0`

17. **HW**:
    - Description: A binary variable indicating the presence of a heatwave on the day.
    - Datatype: `Int64`
    - Example: `0`

18. **cum_HW**:
    - Description: Cumulative number of heatwave days up to the current date.
    - Datatype: `Int64`
    - Example: `0`

19. **MeanTemperature**:
    - Description: The mean temperature recorded on the day.
    - Datatype: `float`
    - Example: `-3.025`

20. **MeanTHI_adj**:
    - Description: The mean adjusted Temperature-Humidity Index for the day.
    - Datatype: `float`
    - Example: `28.012944`

21. **FarmHeatStressMilkProduction**:
    - Description: The relative change in milk production based on farm and heat stress conditions.
    - Datatype: `float`
    - Example: `0.009435`